In [8]:
import pandas as pd
import numpy as np
import tensorflow as tf
import cv2
import mediapipe as mp
import pandas as pd
import numpy as np
# import nbformat
import tensorflow as tf

mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_holistic = mp.solutions.holistic





ROWS_PER_FRAME = 543  # number of landmarks per frame

def load_relevant_data_subset(pq_path):
    data_columns = ['x', 'y', 'z']
    data = pd.read_parquet(pq_path, columns=data_columns)
    n_frames = int(len(data) / ROWS_PER_FRAME)
    data = data.values.reshape(n_frames, ROWS_PER_FRAME, len(data_columns))
    return data.astype(np.float32)

In [3]:
def create_frame_landmark_df(results, frame, xyz):
    xyz_skel = xyz[['type','landmark_index']].drop_duplicates().reset_index(drop=True).copy()

    face = pd.DataFrame()
    pose = pd.DataFrame()
    left_hand = pd.DataFrame()
    right_hand = pd.DataFrame()
    if results.face_landmarks:
        for i, point in enumerate(results.face_landmarks.landmark):
            face.loc[i, ['x','y','z']] = [point.x, point.y, point.z]
    if results.pose_landmarks:
        for i , point in enumerate(results.pose_landmarks.landmark):
            pose.loc[i, ['x', 'y', 'z']] = [point.x, point.y, point.z]
    if results.left_hand_landmarks:
        for i, point in enumerate(results.left_hand_landmarks.landmark):
            left_hand.loc[i, ['x','y','z']] = [point.x, point.y, point.z]
    if results.right_hand_landmarks:
        for i, point in enumerate(results.right_hand_landmarks.landmark):
            right_hand.loc[i, ['x', 'y', 'z']] = [point.x, point.y, point.z]   
    face = face.reset_index() \
        .rename(columns={'index':'landmark_index'}) \
            .assign(type='face')
    pose = pose.reset_index() \
        .rename(columns={'index':'landmark_index'}) \
            .assign(type='pose')
    left_hand = left_hand.reset_index() \
        .rename(columns={'index':'landmark_index'}) \
            .assign(type='left_hand')
    right_hand = right_hand.reset_index() \
        .rename(columns={'index':'landmark_index'}) \
            .assign(type='right_hand')


    landmarks = pd.concat([face,pose,right_hand,left_hand]).reset_index(drop=True)
    landmarks = xyz_skel.merge(landmarks, on=['type','landmark_index'], how='left')
    landmarks = landmarks.assign(frame=frame)
    return landmarks



def do_capture_loop(xyz):
    all_landmarks = []
    cap = cv2.VideoCapture(1)
    with mp_holistic.Holistic(
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5) as holistic:
        frame = 0
        while cap.isOpened():
            frame+=1
            success, image = cap.read()
            if not success:
                print("Ignoring empty camera frame.")
                # If loading a video, use 'break' instead of 'continue'.
                continue

        # To improve performance, optionally mark the image as not writeable to
        # pass by reference.
            image.flags.writeable = False
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            results = holistic.process(image)

        ## create landmarks dataframe from results
            landmarks = create_frame_landmark_df(results,frame,xyz)
            all_landmarks.append(landmarks)
        
        # Draw landmark annotation on the image.
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            mp_drawing.draw_landmarks(
                image,
                results.face_landmarks,
                mp_holistic.FACEMESH_CONTOURS,
                landmark_drawing_spec=None,
                connection_drawing_spec=mp_drawing_styles
                .get_default_face_mesh_contours_style())
            mp_drawing.draw_landmarks(
                image,
                results.pose_landmarks,
                mp_holistic.POSE_CONNECTIONS,
                landmark_drawing_spec=mp_drawing_styles
                .get_default_pose_landmarks_style())
            #Flip the image horizontally for a selfie-view display.
            
            # text = prediction_func(pq_file)
            # print(text)
            # draw_predictions(image,text)
            # cv2.rectangle(frame, (x, y - text_height - 5), (x + text_width, y), (0, 0, 0), -1)
            # cv2.putText(image, text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

            cv2.imshow('MediaPipe Holistic', cv2.flip(image, 1))

            if cv2.waitKey(5) & 0xFF == 27:
                break
    # cap.release()
    return all_landmarks
pq_file_sample = "train_landmark_files/16069/100015657.parquet"
xyz = pd.read_parquet(pq_file_sample)
landmarks = do_capture_loop(xyz)
pd.concat(landmarks).reset_index(drop=True).to_parquet('output.parquet')

In [13]:
import cv2
import mediapipe as mp
import pandas as pd
import numpy as np
# import nbformat
import tensorflow as tf

# def prediction_func(pq_file):

#     ## defining the model 
#     interpreter = tf.lite.Interpreter("model.tflite")
#     found_signatures = list(interpreter.get_signature_list().keys())
#     prediction_fn = interpreter.get_signature_runner("serving_default")

    
#     # Add ordinally Encoded Sign (assign number to each sign name)\
#     train = pd.read_csv('train.csv.zip')
#     train['sign_ord'] = train['sign'].astype('category').cat.codes

#     # Dictionaries to translate sign <-> ordinal encoded sign
#     SIGN2ORD = train[['sign', 'sign_ord']].set_index('sign').squeeze().to_dict()
#     ORD2SIGN = train[['sign_ord', 'sign']].set_index('sign_ord').squeeze().to_dict()

#     ## load data from output parquet
#     xyz_np = load_relevant_data_subset(pq_file)
#     prediction = prediction_fn(inputs=xyz_np)
#     sign = prediction['outputs'].argmax()
#     return ORD2SIGN[sign]
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_holistic = mp.solutions.holistic


def create_frame_landmark_df(results, frame, xyz):
    xyz_skel = xyz[['type','landmark_index']].drop_duplicates().reset_index(drop=True).copy()

    face = pd.DataFrame()
    pose = pd.DataFrame()
    left_hand = pd.DataFrame()
    right_hand = pd.DataFrame()
    if results.face_landmarks:
        for i, point in enumerate(results.face_landmarks.landmark):
            face.loc[i, ['x','y','z']] = [point.x, point.y, point.z]
    if results.pose_landmarks:
        for i , point in enumerate(results.pose_landmarks.landmark):
            pose.loc[i, ['x', 'y', 'z']] = [point.x, point.y, point.z]
    if results.left_hand_landmarks:
        for i, point in enumerate(results.left_hand_landmarks.landmark):
            left_hand.loc[i, ['x','y','z']] = [point.x, point.y, point.z]
    if results.right_hand_landmarks:
        for i, point in enumerate(results.right_hand_landmarks.landmark):
            right_hand.loc[i, ['x', 'y', 'z']] = [point.x, point.y, point.z]   
    face = face.reset_index() \
        .rename(columns={'index':'landmark_index'}) \
            .assign(type='face')
    pose = pose.reset_index() \
        .rename(columns={'index':'landmark_index'}) \
            .assign(type='pose')
    left_hand = left_hand.reset_index() \
        .rename(columns={'index':'landmark_index'}) \
            .assign(type='left_hand')
    right_hand = right_hand.reset_index() \
        .rename(columns={'index':'landmark_index'}) \
            .assign(type='right_hand')


    landmarks = pd.concat([face,pose,right_hand,left_hand]).reset_index(drop=True)
    landmarks = xyz_skel.merge(landmarks, on=['type','landmark_index'], how='left')
    landmarks = landmarks.assign(frame=frame)
    return landmarks


# For webcam input:
def do_capture_loop(xyz):
    all_landmarks = []
    cap = cv2.VideoCapture(1)
    with mp_holistic.Holistic(
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5) as holistic:
        frame = 0
        while cap.isOpened():
            frame+=1
            success, image = cap.read()
            if not success:
                print("Ignoring empty camera frame.")
                # If loading a video, use 'break' instead of 'continue'.
                continue

        # To improve performance, optionally mark the image as not writeable to
        # pass by reference.
            image.flags.writeable = False
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            results = holistic.process(image)

        ## create landmarks dataframe from results
            landmarks = create_frame_landmark_df(results,frame,xyz)
            all_landmarks.append(landmarks)
        
        # Draw landmark annotation on the image.
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            mp_drawing.draw_landmarks(
                image,
                results.face_landmarks,
                mp_holistic.FACEMESH_CONTOURS,
                landmark_drawing_spec=None,
                connection_drawing_spec=mp_drawing_styles
                .get_default_face_mesh_contours_style())
            mp_drawing.draw_landmarks(
                image,
                results.pose_landmarks,
                mp_holistic.POSE_CONNECTIONS,
                landmark_drawing_spec=mp_drawing_styles
                .get_default_pose_landmarks_style())
            #Flip the image horizontally for a selfie-view display.
            
            # text = prediction_func(pq_file)
            # print(text)
            # draw_predictions(image,text)
            # cv2.rectangle(frame, (x, y - text_height - 5), (x + text_width, y), (0, 0, 0), -1)
            # cv2.putText(image, text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

            cv2.imshow('MediaPipe Holistic', cv2.flip(image, 1))

            if cv2.waitKey(5) & 0xFF == 27:
                break
    # cap.release()
    return all_landmarks
pq_file_sample = "train_landmark_files/16069/100015657.parquet"
xyz = pd.read_parquet(pq_file_sample)
landmarks = do_capture_loop(xyz)
pd.concat(landmarks).reset_index(drop=True).to_parquet('output.parquet')

In [9]:
# pq_file = "output.parquet"
def prediction_func(pq_file):

    ## defining the model 
    interpreter = tf.lite.Interpreter("model.tflite")
    found_signatures = list(interpreter.get_signature_list().keys())
    prediction_fn = interpreter.get_signature_runner("serving_default")

    
    # Add ordinally Encoded Sign (assign number to each sign name)\
    train = pd.read_csv('train.csv.zip')
    train['sign_ord'] = train['sign'].astype('category').cat.codes

    # Dictionaries to translate sign <-> ordinal encoded sign
    SIGN2ORD = train[['sign', 'sign_ord']].set_index('sign').squeeze().to_dict()
    ORD2SIGN = train[['sign_ord', 'sign']].set_index('sign_ord').squeeze().to_dict()

    ## load data from output parquet
    xyz_np = load_relevant_data_subset(pq_file)
    prediction = prediction_fn(inputs=xyz_np)
    sign = prediction['outputs'].argmax()
    return ORD2SIGN[sign]






In [11]:
prediction_func(pq_file="output.parquet")
# pd.Series(prediction['outputs']).plot(figsize=(10,4))

'shhh'

In [15]:
# train.query("sign=='shhh'")

,path,participant_id,sequence_id,sign,sign_ord
79,train_landmark_files/37055/1003109377.parquet,37055,1003109377,shhh,194
418,train_landmark_files/18796/1017029978.parquet,18796,1017029978,shhh,194
546,train_landmark_files/18796/1023157153.parquet,18796,1023157153,shhh,194
705,train_landmark_files/18796/1029866550.parquet,18796,1029866550,shhh,194
732,train_landmark_files/4718/1030999521.parquet,4718,1030999521,shhh,194
...,...,...,...,...,...
93588,train_landmark_files/36257/962546588.parquet,36257,962546588,shhh,194
93982,train_landmark_files/37779/979999637.parquet,37779,979999637,shhh,194
94031,train_landmark_files/37055/981603802.parquet,37055,981603802,shhh,194
94376,train_landmark_files/25571/99567121.parquet,25571,99567121,shhh,194
